# Efficient Portfolios
###### Petko Vladimirov Stoyanov - 21208084 - UCD Michael Smurfit Business School

## Part 1 - 40 marks
Analyse two models for the input of historical data.

#### 1. Model 1
Download weekly price of the stock indices [27-Jan-2020, 23-Mar-2020] (9 prices, 8 returns):
    - US: Dow Jones Industrial `^DJI`, S&P500 `^GSPC`
    - UK: FTSE100 `^FTSE`
    - Europe: MSCI Eurozone `EZU`
    - Gold `GLD`

In [504]:
import yfinance as yf
import pandas as pd
import numpy as np

In [486]:

ticker_list = ['^DJI', '^GSPC', '^FTSE', 'EZU', 'GLD']
model1 = yf.download(ticker_list, start="2020-01-27", end="2020-03-23", interval="1wk")['Adj Close']
model1

[*********************100%***********************]  5 of 5 completed


,EZU,GLD,^DJI,^FTSE,^GSPC
Date,,,,,
2020-01-27,38.602436,149.330002,28256.029297,7286.000000,3225.520020
2020-02-03,39.699097,147.789993,29102.509766,7466.700195,3327.709961
2020-02-10,39.994717,149.000000,29398.080078,7409.100098,3380.159912
2020-02-17,39.584667,154.699997,28992.410156,7403.899902,3337.750000
2020-02-24,35.751129,148.380005,25409.359375,6580.600098,2954.219971
2020-03-02,35.293381,157.550003,25864.779297,6462.600098,2972.370117
2020-03-09,29.018581,143.279999,23185.619141,5366.100098,2711.020020
2020-03-16,25.261328,140.110001,19173.980469,5190.799805,2304.919922


#### 2. Model 2
Download weekly price of the stock indices [Jan-2015, Dec-2019]:
    - US: Dow Jones Industrial `^DJI`, S&P500 `^GSPC`
    - UK: FTSE100 `^FTSE`
    - Europe: MSCI Eurozone `EZU`
    - Gold `GLD`

In [432]:
model2 = yf.download(ticker_list, period="5y", start="2015-01-01", interval="1wk")['Adj Close']
model2.dropna(inplace=True)
model2

[*********************100%***********************]  5 of 5 completed


,EZU,GLD,^DJI,^FTSE,^GSPC
Date,,,,,
2014-12-29,30.436686,114.080002,17832.990234,6547.799805,2058.199951
2015-01-05,29.382236,117.260002,17737.369141,6501.100098,2044.810059
2015-01-12,30.177259,122.519997,17511.570312,6550.299805,2019.420044
2015-01-19,30.587320,124.230003,17672.599609,6832.799805,2051.820068
2015-01-26,30.696110,123.449997,17164.949219,6749.399902,1994.989990
...,...,...,...,...,...
2022-01-03,49.369999,167.750000,36231.660156,7485.299805,4677.029785
2022-01-10,49.369999,169.669998,35911.808594,7543.000000,4662.850098
2022-01-17,47.590000,171.089996,34265.371094,7494.100098,4397.939941


#### 3. Calculate expected annual returns and covariance matrix.

In order to calculate the returns of a certain stock we will use the adjusted close price of each period.
Annual (total) returns: $ R_a = \frac{R_n - R_0}{R_0}$

In [492]:
def annual_return(time_series):
    return (time_series[-1]-time_series[0])/time_series[0]

def clear_model(model):
    model_clear = model
    if type(model.index) is not pd.MultiIndex:
        model_clear.index = pd.MultiIndex.from_tuples(zip(model.index.year,model.index),names=['Year','Date'])
    # model_clear.columns = model_clear.columns.droplevel(1)
    return model_clear

def get_returns(model):
    returns = model.groupby('Year').count()
    for ticker in model:
        for year in returns.index:
            returns.loc[year,ticker] = annual_return(model.loc[year,ticker])
    return returns

def get_returns_adjusted(model):
    returns = model.groupby('Year').count()
    for ticker in model:
        for year in returns.index:
            if year-1 in returns.index:
                prev_close = model.loc[year-1,ticker].iloc[-1:]
                annual_series = prev_close.iloc[-1:].append(model.loc[year,ticker])
                returns.loc[year,ticker] = annual_return(annual_series)
            else:
                returns.loc[year,ticker] = annual_return(model.loc[year,ticker])
    return returns


Annual Returns:

In [502]:
models = [model1, model2]

for model in models:
    model_clear = clear_model(model)
    display(get_returns_adjusted(model_clear))
    # display(get_returns_adjusted(model_clear))

,EZU,GLD,^DJI,^FTSE,^GSPC
Year,,,,,
2020,-0.345603,-0.061742,-0.32142,-0.287565,-0.285411


,EZU,GLD,^DJI,^FTSE,^GSPC
Year,,,,,
2014,0.000000,0.000000,0.000000,0.000000,0.000000
2015,-0.017495,-0.110624,-0.022877,-0.046657,-0.006928
2016,0.018633,0.080327,0.134150,0.144258,0.095350
2017,0.278902,0.128091,0.250808,0.076301,0.194200
2018,-0.155695,-0.017873,-0.052027,-0.110617,-0.052988
2019,0.213051,0.201087,0.221981,0.114810,0.277617
2020,0.078226,0.222816,0.068853,-0.152432,0.161126
2021,0.136426,-0.041489,0.187275,0.143023,0.268927
2022,-0.032719,0.002925,-0.042504,0.017461,-0.068315


Log returns and annual covariance matrix:

In [509]:
model1_log = np.log(model1).diff().dropna()  # weekly log returns
cov_matrix1 = model1_log.cov()
cov_matrix1

,EZU,GLD,^DJI,^FTSE,^GSPC
EZU,0.007225,0.003448,0.006419,0.005557,0.005606
GLD,0.003448,0.002642,0.002821,0.003133,0.002350
^DJI,0.006419,0.002821,0.007576,0.003957,0.006624
^FTSE,0.005557,0.003133,0.003957,0.005689,0.003526
^GSPC,0.005606,0.002350,0.006624,0.003526,0.005831


In [510]:
model2_log = np.log(model2).diff().dropna()  # weekly log returns
cov_matrix2 = model2_log.cov()
cov_matrix2

,EZU,GLD,^DJI,^FTSE,^GSPC
EZU,0.000818,0.000095,0.000593,0.000504,0.000551
GLD,0.000095,0.000379,0.000064,0.000049,0.000061
^DJI,0.000593,0.000064,0.000630,0.000413,0.000573
^FTSE,0.000504,0.000049,0.000413,0.000518,0.000382
^GSPC,0.000551,0.000061,0.000573,0.000382,0.000554


#### 4. Optimal portfolio in each case

use 3 month tbl

#### 5. Which model is correct. Why?